<a href="https://colab.research.google.com/github/adriaanslechten/colabs/blob/main/Pose_Classification_Fitwars_simple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview

This Colab helps to create a training set for MediaPipe [Pose Classification](https://google.github.io/mediapipe/solutions/pose_classification.html) soultion, export it to a CSV and then use it in the [ML Kit sample app](https://developers.google.com/ml-kit/vision/pose-detection/classifying-poses#4_integrate_with_the_ml_kit_quickstart_app).

# Step 0: Start Colab

Connect the Colab to hosted Python3 runtime (check top-right corner) and then install required dependencies.

In [ ]:
!pip install numpy==1.19.3
!pip install opencv-python==4.5.1.48
!pip install tqdm==4.56.0

!pip install mediapipe==0.8.3

     |████████████████████████████████| 14.9 MB 94 kB/s 
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


     |████████████████████████████████| 50.4 MB 15 kB/s 
  Attempting uninstall: opencv-python
    Found existing installation: opencv-python 4.1.2.30
    Uninstalling opencv-python-4.1.2.30:
      Successfully uninstalled opencv-python-4.1.2.30
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.


     |████████████████████████████████| 72 kB 888 kB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.62.0
    Uninstalling tqdm-4.62.0:
      Successfully uninstalled tqdm-4.62.0


     |████████████████████████████████| 67.0 MB 21 kB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Step 1: Upload image samples

Locally create a folder named `fitness_poses_images_in` with image samples.

Images should repesent terminal states of desired pose classes. I.e. if you want to classify push-up provide iamges for two classes: when person is up, and when person is down.

There should be about a few hundred samples per class covering different camera angles, environment conditions, body shapes, and exercise variations to build a good classifier.

Required structure of the images_in_folder:
```
fitness_poses_images_in/
  pushups_up/
    image_001.jpg
    image_002.jpg
    ...
  pushups_down/
    image_001.jpg
    image_002.jpg
    ...
  ...
```

In [ ]:
ls drive/MyDrive/fitness_poses_images_in

down/  up/


# Step 2: Create samples for classifier

Runs BlazePose on provided images to get target poses for the classifier in a format required by the demo App.

In [57]:


import csv
import cv2
import numpy as np
import os
import sys
import tqdm

from mediapipe.python.solutions import drawing_utils as mp_drawing
from mediapipe.python.solutions import pose as mp_pose
from itertools import chain
import json

JSON_OUT_PATH = "fitness_poses_out_basic.json"
IMAGES_IN_FOLDER = "drive/MyDrive/fitness_poses_images_in"
IMAGES_OUT_FOLDER = "fitness_poses_images_out_basic"


def get_class_names():
    """Folder names are used as pose class names."""
    return sorted([n for n in os.listdir(images_in_folder) if not n.startswith(".")])


def make_dirs(images_out_folder, pose_class_name):
    if not os.path.exists(os.path.join(images_out_folder, pose_class_name)):
        os.makedirs(os.path.join(images_out_folder, pose_class_name))


def get_image_name(images_in_folder, pose_class_name):
    return sorted([n for n in os.listdir(os.path.join(images_in_folder, pose_class_name)) if not n.startswith(".")])


def load_image(images_in_folder, pose_class_name, image_name):
    return cv2.cvtColor(cv2.imread(os.path.join(images_in_folder, pose_class_name, image_name)), cv2.COLOR_BGR2RGB)


def get_landmarks(input_frame):
    result = pose_tracker.process(image=input_frame)
    return result.pose_landmarks


def save_image():
    output_frame = input_frame.copy()
    mp_drawing.draw_landmarks(image=output_frame, landmark_list=pose_landmarks, connections=mp_pose.POSE_CONNECTIONS)
    output_frame = cv2.cvtColor(output_frame, cv2.COLOR_RGB2BGR)
    cv2.imwrite(os.path.join(images_out_folder, image_name), output_frame)
    return output_frame


def map_to_abs_coords(pose_landmarks, output_frame):
    """Map pose landmarks from [0, 1] range to absolute coordinates to get
    correct aspect ratio."""
    pose_landmarks = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]
    frame_height, frame_width = output_frame.shape[:2]
    pose_landmarks *= np.array([frame_width, frame_height, frame_width])
    return [
        {f"x{idx}": row[0], f"y{idx}": row[1], f"z{idx}": row[2]} for idx, row in enumerate(pose_landmarks.tolist())
    ]


def make_list(pose_landmarks):
    return np.around(pose_landmarks, 5).flatten().astype(np.str).tolist()


def create_flattened_dict(pose_landmarks, pose_class_name):
    final_landmarks = {}
    for landmark in pose_landmarks:
        final_landmarks.update(landmark)
    final_landmarks["output"] = pose_class_name
    return final_landmarks


aggregated_output = []
for pose_class_name in get_class_names():
    print("Bootstrapping ", pose_class_name, file=sys.stderr)
    make_dirs(IMAGES_OUT_FOLDER, pose_class_name)
    image_names = get_image_name(IMAGES_IN_FOLDER, pose_class_name)
    for image_name in tqdm.tqdm(image_names, position=0):
        # Load image.
        input_frame = load_image(IMAGES_IN_FOLDER, pose_class_name, image_name)
        # Initialize fresh pose tracker and run it.
        with mp_pose.Pose(upper_body_only=False) as pose_tracker:
            pose_landmarks = get_landmarks(input_frame)
            if pose_landmarks is not None:  # Save image with pose prediction (if pose was detected).
                assert len(pose_landmarks.landmark) == 33, "Unexpected number of predicted pose landmarks: {}".format(
                    len(pose_landmarks.landmark)
                )
                output_frame = save_image()
                pose_landmarks = map_to_abs_coords(pose_landmarks, output_frame)
                final_landmarks = create_flattened_dict(pose_landmarks, pose_class_name)
                aggregated_output.append(final_landmarks)


with open(JSON_OUT_PATH, "w") as outfile:
    json.dump(aggregated_output, outfile)

print(aggregated_output)

Bootstrapping  down
100%|██████████| 52/52 [00:06<00:00,  7.46it/s]
Bootstrapping  up
100%|██████████| 89/89 [00:11<00:00,  7.51it/s]

[{'x0': 397.74792194366455, 'y0': 315.0692582130432, 'z0': -495.1861381530762, 'x1': 404.0402412414551, 'y1': 296.7873752117157, 'z1': -479.9308776855469, 'x2': 410.18409729003906, 'y2': 295.4211115837097, 'z2': -479.9576759338379, 'x3': 416.33167266845703, 'y3': 294.0016508102417, 'z3': -479.8718452453613, 'x4': 388.4075403213501, 'y4': 299.9295651912689, 'z4': -477.32577323913574, 'x5': 382.236909866333, 'y5': 300.68103075027466, 'z5': -477.29363441467285, 'x6': 376.0652542114258, 'y6': 301.5259265899658, 'z6': -477.3128032684326, 'x7': 425.8871555328369, 'y7': 297.020423412323, 'z7': -375.59974193573, 'x8': 371.30911350250244, 'y8': 307.00693130493164, 'z8': -354.89110946655273, 'x9': 412.0861053466797, 'y9': 325.61609745025635, 'z9': -453.53455543518066, 'x10': 389.6677255630493, 'y10': 329.4301986694336, 'z10': -448.0886936187744, 'x11': 464.9401664733887, 'y11': 368.88842582702637, 'z11': -288.5856628417969, 'x12': 336.92688941955566, 'y12': 367.4745798110962, 'z12': -288.6960506